# Build Docker image 

In this notebook, we will build the docker container that contains the ResNet152 model, Flask web application, model driver and all dependencies.
Make sure you have logged in using docker login.

In [1]:
import os
from os import path
import json
import shutil

In [2]:
os.makedirs('flaskwebapp', exist_ok=True)
os.makedirs(os.path.join('flaskwebapp', 'nginx'), exist_ok=True)
os.makedirs(os.path.join('flaskwebapp', 'etc'), exist_ok=True)

In [3]:
shutil.copy('resnet152.py', 'flaskwebapp')
shutil.copy('driver.py', 'flaskwebapp')
os.listdir('flaskwebapp')

['driver.py', 'resnet152.py', 'etc', 'nginx']

Below, we create the module for the Flask web application.

In [4]:
%%writefile flaskwebapp/app.py

from flask import Flask, request
import logging
import json
import driver

app = Flask(__name__)
predict_for = driver.get_model_api()
 
@app.route("/score", methods = ['POST'])
def scoreRRS():
    """ Endpoint for scoring
    """
    if request.headers['Content-Type'] != 'application/json':
        return Response(json.dumps({}), status= 415, mimetype ='application/json')
    request_input = request.json['input']
    response = predict_for(request_input)
    print(response)
    return json.dumps({'result': str(response)})


@app.route("/")
def healthy():
    return "Healthy"

# Tensorflow Version
@app.route('/version', methods = ['GET'])
def version_request():
    return driver.version()

if __name__ == "__main__":
    app.run(host='0.0.0.0', port=5000)

Writing flaskwebapp/app.py


In [5]:
%%writefile flaskwebapp/wsgi.py
from app import app as application

def create():
    print("Initialising")
    application.run(host='127.0.0.1', port=5000)

Writing flaskwebapp/wsgi.py


Here, we write the configuration for the Nginx which creates a proxy between ports **80** and **5000**.

In [6]:
%%writefile flaskwebapp/nginx/app
server {
    listen 80;
    server_name _;
 
    location / {
    include proxy_params;
    proxy_pass http://127.0.0.1:5000;
    proxy_connect_timeout 5000s;
    proxy_read_timeout 5000s;
  }
}

Writing flaskwebapp/nginx/app


In [7]:
%%writefile flaskwebapp/gunicorn_logging.conf

[loggers]
keys=root, gunicorn.error

[handlers]
keys=console

[formatters]
keys=json

[logger_root]
level=INFO
handlers=console

[logger_gunicorn.error]
level=ERROR
handlers=console
propagate=0
qualname=gunicorn.error

[handler_console]
class=StreamHandler
formatter=json
args=(sys.stdout, )

[formatter_json]
class=jsonlogging.JSONFormatter

Writing flaskwebapp/gunicorn_logging.conf


In [8]:
%%writefile flaskwebapp/kill_supervisor.py
import sys
import os
import signal

def write_stdout(s):
    sys.stdout.write(s)
    sys.stdout.flush()

# this function is modified from the code and knowledge found here: http://supervisord.org/events.html#example-event-listener-implementation
def main():
    while 1:
        write_stdout('READY\n')
        # wait for the event on stdin that supervisord will send
        line = sys.stdin.readline()
        write_stdout('Killing supervisor with this event: ' + line);
        try:
            # supervisord writes its pid to its file from which we read it here, see supervisord.conf
            pidfile = open('/tmp/supervisord.pid','r')
            pid = int(pidfile.readline());
            os.kill(pid, signal.SIGQUIT)
        except Exception as e:
            write_stdout('Could not kill supervisor: ' + e.strerror + '\n')
            write_stdout('RESULT 2\nOK')

main()

Writing flaskwebapp/kill_supervisor.py


In [9]:
%%writefile flaskwebapp/etc/supervisord.conf 
[supervisord]
logfile=/tmp/supervisord.log ; (main log file;default $CWD/supervisord.log)
logfile_maxbytes=50MB        ; (max main logfile bytes b4 rotation;default 50MB)
logfile_backups=10           ; (num of main logfile rotation backups;default 10)
loglevel=info                ; (log level;default info; others: debug,warn,trace)
pidfile=/tmp/supervisord.pid ; (supervisord pidfile;default supervisord.pid)
nodaemon=true                ; (start in foreground if true;default false)
minfds=1024                  ; (min. avail startup file descriptors;default 1024)
minprocs=200                 ; (min. avail process descriptors;default 200)

[program:gunicorn]
command=bash -c "gunicorn --workers 1 -m 007 --timeout 100000 --capture-output --error-logfile - --log-level debug --log-config gunicorn_logging.conf \"wsgi:create()\""
directory=/code
redirect_stderr=true
stdout_logfile =/dev/stdout
stdout_logfile_maxbytes=0
startretries=2
startsecs=20

[program:nginx]
command=/usr/sbin/nginx -g "daemon off;"
startretries=2
startsecs=5
priority=3

[eventlistener:program_exit]
command=python kill_supervisor.py
directory=/code
events=PROCESS_STATE_FATAL
priority=2

Writing flaskwebapp/etc/supervisord.conf


We create a custom image based on the CUDA 8 image from NVIDIA and install all the necessary dependencies. This is in order to try and keep the size of the image as small as possible.

In [10]:
%%writefile flaskwebapp/requirements.txt

Pillow==5.0.0
click==6.7
configparser==3.5.0
Flask==0.12.2
gunicorn==19.6.0
json-logging-py==0.2
MarkupSafe==1.0
olefile==0.44
requests==2.18.4
h5py==2.6.0

Writing flaskwebapp/requirements.txt


In [11]:
%%writefile flaskwebapp/dockerfile

FROM nvidia/cuda:8.0-cudnn6-devel-ubuntu16.04
    
RUN echo "deb http://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1604/x86_64 /" > /etc/apt/sources.list.d/nvidia-ml.list

USER root
RUN mkdir /code
WORKDIR /code
RUN chmod -R a+w /code
ADD . /code/

RUN apt-get update && apt-get install -y --no-install-recommends \
        build-essential \
        ca-certificates \
        cmake \
        wget \
        curl \
        git \
        supervisor \
        nginx && \
    rm -rf /var/lib/apt/lists/* &&\
    rm /etc/nginx/sites-enabled/default && \
    cp /code/nginx/app /etc/nginx/sites-available/ && \
    ln -s /etc/nginx/sites-available/app /etc/nginx/sites-enabled/ 

ENV PYTHON_VERSION=3.5
RUN curl -o ~/miniconda.sh -O  https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh  && \
    chmod +x ~/miniconda.sh && \
    ~/miniconda.sh -b -p /opt/conda && \
    rm ~/miniconda.sh && \
    /opt/conda/bin/conda create -y --name py$PYTHON_VERSION python=$PYTHON_VERSION numpy pyyaml scipy \
    ipython pandas jupyter ipykernel scikit-learn && \
    /opt/conda/bin/conda clean -ya
    
ENV PATH /opt/conda/envs/py$PYTHON_VERSION/bin:$PATH
ENV LD_LIBRARY_PATH /opt/conda/envs/py$PYTHON_VERSION/lib:/usr/local/cuda/lib64/:$LD_LIBRARY_PATH
    
RUN pip install --upgrade pip && \
    pip install tensorflow-gpu==1.4.1 && \
    pip install keras==2.1.5 && \
    pip install -r /code/requirements.txt && \       
    /opt/conda/bin/conda clean -yt

EXPOSE 8888
EXPOSE 5000
EXPOSE 80

CMD ["supervisord", "-c", "/code/etc/supervisord.conf"]

Writing flaskwebapp/dockerfile


The image name below refers to our dockerhub account. If you wish to push the image to your account make sure you change the docker login.

In [12]:
docker_login = 'fboylu'
image_name = docker_login + '/kerastf-gpu'
application_path = 'flaskwebapp'
docker_file_location = path.join(application_path, 'dockerfile')

In [13]:
!docker build -t $image_name -f $docker_file_location $application_path --no-cache

Sending build context to Docker daemon  33.79kB
Step 1/17 : FROM nvidia/cuda:8.0-cudnn6-devel-ubuntu16.04
 ---> 8d377158a37d
Step 2/17 : RUN echo "deb http://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1604/x86_64 /" > /etc/apt/sources.list.d/nvidia-ml.list
 ---> Running in 4cb638e69d29
Removing intermediate container 4cb638e69d29
 ---> ce5b714a253e
Step 3/17 : USER root
 ---> Running in 7850cbe5fe7b
Removing intermediate container 7850cbe5fe7b
 ---> f98325d63b19
Step 4/17 : RUN mkdir /code
 ---> Running in 6d4b81313706
Removing intermediate container 6d4b81313706
 ---> cd10e890dbf0
Step 5/17 : WORKDIR /code
Removing intermediate container 0e25fe61bbb2
 ---> 72731dd63a5d
Step 6/17 : RUN chmod -R a+w /code
 ---> Running in 4a6e0c207bc4
Removing intermediate container 4a6e0c207bc4
 ---> 31e7669bb00f
Step 7/17 : ADD . /code/
 ---> 3eedc803d516
Step 8/17 : RUN apt-get update && apt-get install -y --no-install-recommends         build-essential         ca-certificates

Get:25 http://archive.ubuntu.com/ubuntu xenial/main amd64 libxcb1 amd64 1.11.1-1ubuntu1 [40.0 kB]
Get:26 http://archive.ubuntu.com/ubuntu xenial/main amd64 libx11-data all 2:1.6.3-1ubuntu2 [113 kB]
Get:27 http://archive.ubuntu.com/ubuntu xenial/main amd64 libx11-6 amd64 2:1.6.3-1ubuntu2 [571 kB]
Get:28 http://archive.ubuntu.com/ubuntu xenial-updates/main amd64 wget amd64 1.17.1-1ubuntu1.4 [299 kB]
Get:29 http://archive.ubuntu.com/ubuntu xenial-updates/main amd64 curl amd64 7.47.0-1ubuntu2.8 [139 kB]
Get:30 http://archive.ubuntu.com/ubuntu xenial/main amd64 fonts-dejavu-core all 2.35-1 [1039 kB]
Get:31 http://archive.ubuntu.com/ubuntu xenial-updates/main amd64 fontconfig-config all 2.11.94-0ubuntu1.1 [49.9 kB]
Get:32 http://archive.ubuntu.com/ubuntu xenial/main amd64 liberror-perl all 0.17-1.2 [19.6 kB]
Get:33 http://archive.ubuntu.com/ubuntu xenial-updates/main amd64 git-man all 1:2.7.4-0ubuntu1.4 [736 kB]
Get:34 http://archive.ubuntu.com/ubuntu xenial-updates/main amd64 git amd64 1:2.

Selecting previously unselected package liberror-perl.
Preparing to unpack .../liberror-perl_0.17-1.2_all.deb ...
Unpacking liberror-perl (0.17-1.2) ...
Selecting previously unselected package git-man.
Preparing to unpack .../git-man_1%3a2.7.4-0ubuntu1.4_all.deb ...
Unpacking git-man (1:2.7.4-0ubuntu1.4) ...
Selecting previously unselected package git.
Preparing to unpack .../git_1%3a2.7.4-0ubuntu1.4_amd64.deb ...
Unpacking git (1:2.7.4-0ubuntu1.4) ...
Selecting previously unselected package libfreetype6:amd64.
Preparing to unpack .../libfreetype6_2.6.1-0.1ubuntu2.3_amd64.deb ...
Unpacking libfreetype6:amd64 (2.6.1-0.1ubuntu2.3) ...
Selecting previously unselected package libfontconfig1:amd64.
Preparing to unpack .../libfontconfig1_2.11.94-0ubuntu1.1_amd64.deb ...
Unpacking libfontconfig1:amd64 (2.11.94-0ubuntu1.1) ...
Selecting previously unselected package libjpeg8:amd64.
Preparing to unpack .../libjpeg8_8c-2ubuntu8_amd64.deb ...
Unpacking libjpeg8:amd64 (8c-2ubuntu8) ...
Selecting p

libgfortran-ng-7.2.0 |  1.2 MB | ########## | 100% 
numpy-base-1.14.5    |  4.1 MB | ########## | 100% 
decorator-4.3.0      |   15 KB | ########## | 100% 
markupsafe-1.0       |   25 KB | ########## | 100% 
widgetsnbextension-3 |  1.7 MB | ########## | 100% 
qtconsole-4.3.1      |  151 KB | ########## | 100% 
blas-1.0             |    6 KB | ########## | 100% 
nbformat-4.4.0       |  138 KB | ########## | 100% 
zeromq-4.2.5         |  567 KB | ########## | 100% 
nbconvert-5.3.1      |  397 KB | ########## | 100% 
jupyter_core-4.4.0   |   61 KB | ########## | 100% 
pcre-8.42            |  251 KB | ########## | 100% 
ptyprocess-0.6.0     |   23 KB | ########## | 100% 
pandoc-2.2.1         | 21.0 MB | ########## | 100% 
gstreamer-1.14.0     |  3.8 MB | ########## | 100% 
libpng-1.6.34        |  334 KB | ########## | 100% 
ipykernel-4.8.2      |  146 KB | ########## | 100% 
python-dateutil-2.7. |  261 KB | ########## | 100% 
jupyter-1.0.0        |    5 KB | ########## | 100% 
pandocfilter

Executing transaction: ...working... done

#
# To activate this environment, use:
# > source activate py3.5
#
# To deactivate an active environment, use:
# > source deactivate
#

Cache location: /opt/conda/pkgs
Will remove the following tarballs:

/opt/conda/pkgs
---------------
openssl-1.0.2o-h20670df_0.tar.bz2            3.4 MB
certifi-2018.4.16-py35_0.tar.bz2             143 KB
ruamel_yaml-0.15.37-py36h14c3975_2.tar.bz2     245 KB
cffi-1.11.5-py36h9745a5d_0.tar.bz2           212 KB
sqlite-3.24.0-h84994c4_0.tar.bz2             1.8 MB
readline-7.0-ha6073c6_4.tar.bz2              1.1 MB
asn1crypto-0.24.0-py36_0.tar.bz2             155 KB
pandocfilters-1.4.2-py35h1565a15_1.tar.bz2      12 KB
wcwidth-0.1.7-py35hcd08066_0.tar.bz2          25 KB
jsonschema-2.6.0-py35h4395190_0.tar.bz2       63 KB
urllib3-1.22-py36hbe7ace6_0.tar.bz2          155 KB
ipython-6.4.0-py35_0.tar.bz2                 1.0 MB
send2trash-1.5.0-py35_0.tar.bz2               16 KB
conda-env-2.6.0-h36134e3_1.tar.bz2      

Removed jupyter_client-5.2.3-py35_0.tar.bz2
Removed fontconfig-2.13.0-h9420a91_0.tar.bz2
Removed yaml-0.1.7-had09818_2.tar.bz2
Removed mistune-0.8.3-py35h14c3975_1.tar.bz2
Removed conda-4.5.4-py36_0.tar.bz2
Removed pyopenssl-18.0.0-py36_0.tar.bz2
Removed jpeg-9b-h024ee3a_2.tar.bz2
Removed gst-plugins-base-1.14.0-hbbd80ab_1.tar.bz2
Removed libxml2-2.9.8-h26e45fe_1.tar.bz2
Removed mkl_random-1.0.1-py35h629b387_0.tar.bz2
Removed decorator-4.3.0-py35_0.tar.bz2
Removed glib-2.56.1-h000015b_0.tar.bz2
Removed jupyter_core-4.4.0-py35ha89e94b_0.tar.bz2
Removed entrypoints-0.2.3-py35h48174a2_2.tar.bz2
Removed pytz-2018.5-py35_0.tar.bz2
Removed pyzmq-17.0.0-py35h14c3975_0.tar.bz2
Removed expat-2.2.5-he0dffb1_0.tar.bz2
Removed intel-openmp-2018.0.3-0.tar.bz2
Removed six-1.11.0-py36h372c433_1.tar.bz2
Removed zeromq-4.2.5-hf484d3e_0.tar.bz2
Removed libuuid-1.0.3-h1bed415_2.tar.bz2
Removed certifi-2018.4.16-py36_0.tar.bz2
Removed ipywidgets-7.2.1-py35_0.tar.bz2
Removed ipython_genutils-0.2.0-py35hc9e

mkl-random 1.0.1 requires cython, which is not installed.
mkl-fft 1.0.0 requires cython, which is not installed.
  Running setup.py bdist_wheel for configparser: started
  Running setup.py bdist_wheel for configparser: finished with status 'done'
  Stored in directory: /root/.cache/pip/wheels/a3/61/79/424ef897a2f3b14684a7de5d89e8600b460b89663e6ce9d17c
  Running setup.py bdist_wheel for json-logging-py: started
  Running setup.py bdist_wheel for json-logging-py: finished with status 'done'
  Stored in directory: /root/.cache/pip/wheels/0d/2e/1c/c638b7589610d8b9358a6e5eb008edacb8b3e9b6d1edc9479f
  Running setup.py bdist_wheel for olefile: started
  Running setup.py bdist_wheel for olefile: finished with status 'done'
  Stored in directory: /root/.cache/pip/wheels/c4/19/76/61fc7929d808e51567aff23036ca5fe6ba8336ad0559ca6a27
  Running setup.py bdist_wheel for itsdangerous: started
  Running setup.py bdist_wheel for itsdangerous: finished with status 'done'
  Stored in directory: /root/.cach

In [14]:
!docker push $image_name

The push refers to repository [docker.io/fboylu/kerastf-gpu]

5fb6c494: Preparing 
2cf4aff8: Preparing 
30079cf0: Preparing 
4306dc9b: Preparing 
457b34e8: Preparing 
b6632be8: Preparing 
6701c6e5: Preparing 
c91adc73: Preparing 
1a7e8369: Preparing 
fdf34542: Preparing 
d2de5963: Preparing 
3c5682a5: Preparing 
7dd0a728: Preparing 
91e51d73: Preparing 
d9e65295: Preparing 
45e78935: Preparing 
1dc646ba: Preparing 
denied: requested access to the resource is denied


In [15]:
print('Docker image name {}'.format(image_name))

Docker image name fboylu/kerastf-gpu


We can now [test our image locally](03_TestLocally.ipynb).